In [1]:
import pandas as pd
import numpy as np

In [2]:
demographics = pd.read_csv("dataset_outcomes_demgoraphics.csv")
venti = pd.read_csv("merged_ventilation_antibiotics.csv")


demographics.head()

,Unnamed: 0,hadm_id,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,kdigo_emar_diff_hr,admission_type,insurance,language,marital_status,race,gender,hospital_expire_flag,los_hospital,admission_age
0,0,20001305.0,16003661-115,16003661,2178-03-25 04:48:00,0,14.3,2178-03-27 02:12:00,1.2,1,45.400000,OBSERVATION ADMIT,Medicare,ENGLISH,SINGLE,WHITE,F,1,2,84
1,1,20001305.0,16003661-176,16003661,2178-03-26 09:20:00,500,14.3,2178-03-27 02:12:00,1.2,1,16.866667,OBSERVATION ADMIT,Medicare,ENGLISH,SINGLE,WHITE,F,1,2,84
2,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000,5.2,2158-05-20 12:10:00,1.2,2,1.283333,OBSERVATION ADMIT,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,F,0,21,66
3,3,20011505.0,14104652-176,14104652,2158-05-21 12:40:00,1000,14.7,2158-05-22 04:04:00,0.7,0,15.400000,OBSERVATION ADMIT,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,F,0,21,66
4,4,20011505.0,14104652-186,14104652,2158-05-22 00:11:00,1000,14.7,2158-05-23 14:59:00,0.7,0,38.800000,OBSERVATION ADMIT,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,F,0,21,66


In [3]:
time_bounds = (
    venti.groupby("subject_id_x")
    .agg({"starttime_y": "min", "endtime": "max"})
    .reset_index()
)
time_bounds

# change column name for subject_id_x to subject_id
time_bounds.rename(columns={"subject_id_x": "subject_id"}, inplace=True)
time_bounds

,subject_id,starttime_y,endtime
0,10001884,2131-01-11 04:00:00,2131-01-14 07:00:00
1,10002428,2156-04-19 20:00:00,2156-05-20 09:00:00
2,10003400,2137-02-25 23:37:00,2137-08-20 08:00:00
3,10004235,2196-02-24 17:00:00,2196-02-27 16:48:00
4,10004401,2144-01-27 20:00:00,2144-06-18 11:00:00
...,...,...,...
14002,19997752,2128-03-02 13:54:00,2128-03-02 23:00:00
14003,19997843,2120-11-18 03:00:00,2120-11-19 03:00:00
14004,19998330,2178-10-01 08:08:00,2178-11-29 08:00:00
14005,19999840,2164-09-12 09:27:00,2164-09-17 12:00:00


In [4]:
# Step 1: Merge demographics and time_bounds on subject_id
merged_df = pd.merge(demographics, time_bounds, on="subject_id", how="left")

# Step 2: Filter the merged DataFrame
filtered_df = merged_df[
    (merged_df["emar_charttime"] >= merged_df["starttime_y"])
    & (merged_df["emar_charttime"] <= merged_df["endtime"])
]

In [5]:
filtered_df

,Unnamed: 0,hadm_id,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,...,insurance,language,marital_status,race,gender,hospital_expire_flag,los_hospital,admission_age,starttime_y,endtime
2,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000,5.2,2158-05-20 12:10:00,1.2,2,...,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,F,0,21,66,2158-05-19 18:00:00,2158-05-20 14:00:00
15,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500,17.8,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
16,16,20024229.0,16779215-136,16779215,2132-08-31 00:37:00,1500,24.3,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
17,17,20024229.0,16779215-145,16779215,2132-08-31 07:56:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
18,18,20024229.0,16779215-164,16779215,2132-08-31 16:00:00,1500,24.3,2132-09-02 13:45:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13108,13108,25265899.0,13470702-63,13470702,2134-02-11 03:27:00,1000,8.0,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
13109,13109,25265899.0,13470702-81,13470702,2134-02-11 12:28:00,1000,9.2,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
13110,13110,25265899.0,13470702-97,13470702,2134-02-11 20:15:00,1000,9.2,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
13127,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000,11.8,2150-12-07 03:56:00,4.6,3,...,Other,ENGLISH,NaN,UNKNOWN,M,1,2,23,2150-12-05 04:00:00,2150-12-08 12:00:00


In [6]:
# save the filtered_df to a csv file

filtered_df.to_csv("filtered_final.csv", index=False)

In [7]:
filtered_df

,Unnamed: 0,hadm_id,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,...,insurance,language,marital_status,race,gender,hospital_expire_flag,los_hospital,admission_age,starttime_y,endtime
2,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000,5.2,2158-05-20 12:10:00,1.2,2,...,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,F,0,21,66,2158-05-19 18:00:00,2158-05-20 14:00:00
15,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500,17.8,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
16,16,20024229.0,16779215-136,16779215,2132-08-31 00:37:00,1500,24.3,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
17,17,20024229.0,16779215-145,16779215,2132-08-31 07:56:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
18,18,20024229.0,16779215-164,16779215,2132-08-31 16:00:00,1500,24.3,2132-09-02 13:45:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13108,13108,25265899.0,13470702-63,13470702,2134-02-11 03:27:00,1000,8.0,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
13109,13109,25265899.0,13470702-81,13470702,2134-02-11 12:28:00,1000,9.2,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
13110,13110,25265899.0,13470702-97,13470702,2134-02-11 20:15:00,1000,9.2,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
13127,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000,11.8,2150-12-07 03:56:00,4.6,3,...,Other,ENGLISH,NaN,UNKNOWN,M,1,2,23,2150-12-05 04:00:00,2150-12-08 12:00:00


In [8]:
# only get the unique subject_ids from the filtered_df

unique_subject_ids = filtered_df["subject_id"].unique()
len(unique_subject_ids)

1277

Adding Predictor variables (vitals etc)

Urine

In [9]:
# #order by charttime and stay_id
# urine = urine.sort_values(by=['charttime', 'stay_id'], ascending=True)

# urine[urine['stay_id'] == 33238792].head(50)

In [10]:
urine = pd.read_csv("urine_output.csv")

# groupby urine stay_id and sum the value of the urineoutput

urine_grouped = urine.groupby("stay_id").agg({"urineoutput": "sum"}).reset_index()
urine_grouped

,stay_id,urineoutput
0,30000153,2180.0
1,30000213,4445.0
2,30000484,2640.0
3,30000646,21320.0
4,30001148,1785.0
...,...,...
70359,39999301,600.0
70360,39999384,1325.0
70361,39999552,4273.0
70362,39999562,8575.0


In [11]:
vitals = pd.read_csv("vitalsigns.csv")

In [12]:
vitals = vitals.rename(columns={"charttime": "vital_charttime"})

vitals

,hadm_id,subject_id,stay_id,vital_charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,temperature_site,spo2,glucose
0,24607734,16255318,35069090,2146-03-22 18:00:00,99.0,127.0,79.5,92.0,123.0,60.0,74.0,34.0,NaN,NaN,100.0,NaN
1,20161250,17601166,35276354,2175-11-23 00:00:00,82.0,177.0,89.0,107.0,177.0,89.0,107.0,36.0,37.44,Oral,96.0,123.0
2,28298251,15944907,33478333,2156-01-31 21:09:00,NaN,133.0,110.0,115.0,133.0,110.0,115.0,7.0,NaN,NaN,NaN,NaN
3,25183055,13964474,36812880,2164-03-21 09:00:00,124.0,128.0,85.0,94.0,128.0,85.0,94.0,38.0,NaN,NaN,NaN,NaN
4,24853638,18933099,34994611,2141-09-05 15:00:00,73.0,171.0,85.0,119.0,NaN,NaN,NaN,12.5,NaN,NaN,93.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745495,24739503,12439733,34043429,2113-05-24 03:00:00,116.0,119.0,83.0,92.0,119.0,83.0,92.0,32.0,NaN,NaN,96.0,NaN
9745496,29355167,17414720,39964246,2117-05-31 15:00:00,90.0,145.0,83.0,99.0,145.0,83.0,99.0,32.0,NaN,NaN,96.0,NaN
9745497,23684845,13054680,34153301,2140-11-12 19:00:00,117.0,145.0,83.0,98.0,145.0,83.0,98.0,32.0,NaN,NaN,100.0,NaN
9745498,20972164,10754405,38037508,2165-09-12 22:30:00,117.0,157.0,83.0,100.0,157.0,83.0,100.0,32.0,NaN,NaN,96.0,NaN


In [13]:
# only keep vitals information of the unique subject_ids

# vitals = vitals[vitals['stay_id'].isin(unique_subject_ids)]

vitals[vitals["subject_id"] == 14104652]

,hadm_id,subject_id,stay_id,vital_charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,temperature_site,spo2,glucose
19459,24403810,14104652,35461437,2158-03-20 03:00:00,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.0,NaN
59366,24403810,14104652,35461437,2158-03-20 04:01:00,NaN,115.0,67.0,82.0,115.0,67.0,82.0,NaN,NaN,NaN,NaN,NaN
97161,20011505,14104652,39090944,2158-05-20 10:00:00,92.0,119.0,67.0,84.0,NaN,NaN,NaN,26.5,NaN,NaN,100.0,NaN
113764,20011505,14104652,39090944,2158-05-19 18:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,NaN
241825,20011505,14104652,39090944,2158-05-22 21:00:00,98.0,122.0,73.0,87.0,122.0,73.0,87.0,24.0,NaN,NaN,96.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9629638,20011505,14104652,39090944,2158-05-24 11:00:00,85.0,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,100.0,NaN
9672794,20011505,14104652,39090944,2158-05-19 20:00:00,143.0,100.0,69.0,80.0,NaN,NaN,NaN,32.0,NaN,NaN,100.0,NaN
9695427,20011505,14104652,39090944,2158-05-21 03:00:00,78.0,104.0,55.0,72.0,NaN,NaN,NaN,19.0,NaN,NaN,96.0,NaN
9703832,20011505,14104652,39090944,2158-05-22 17:00:00,81.0,110.0,66.0,77.0,110.0,66.0,77.0,20.0,36.78,Oral,96.0,NaN


In [14]:
vitals_filtered = vitals[vitals["subject_id"].isin(unique_subject_ids)]

In [15]:
# order vitals_filtered by charttime and stay_id

vitals_filtered = vitals_filtered.sort_values(
    by=["vital_charttime", "stay_id"], ascending=True
)

# filter if one of heart_rate or sbp or glucose or temperature is null

vitals_filtered = vitals_filtered.dropna(
    subset=["heart_rate", "sbp", "glucose", "temperature", "dbp_ni"], how="all"
)

In [16]:
vitals_filtered[vitals_filtered["stay_id"] == 30927169]

,hadm_id,subject_id,stay_id,vital_charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,temperature_site,spo2,glucose
1906746,22277686,16006168,30927169,2110-01-21 06:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.0
2680406,22277686,16006168,30927169,2110-01-21 07:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0
3827716,22277686,16006168,30927169,2110-01-21 09:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0
1483998,22277686,16006168,30927169,2110-01-21 12:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0
3442309,22277686,16006168,30927169,2110-01-21 13:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954183,22277686,16006168,30927169,2110-01-25 18:02:00,NaN,133.0,73.0,84.0,133.0,73.0,84.0,NaN,NaN,NaN,NaN,NaN
1050406,22277686,16006168,30927169,2110-01-25 19:00:00,82.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,97.0,87.0
1958230,22277686,16006168,30927169,2110-01-25 19:02:00,NaN,133.0,58.0,76.0,133.0,58.0,76.0,NaN,NaN,NaN,NaN,NaN
191844,22277686,16006168,30927169,2110-01-25 20:00:00,83.0,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,100.0,NaN


In [17]:
# Step 1: Convert vital_charttime to datetime and set it as the index
vitals_filtered_1 = vitals_filtered
vitals_filtered_1["vital_charttime"] = pd.to_datetime(
    vitals_filtered_1["vital_charttime"]
)
vitals_filtered_1 = vitals_filtered_1.set_index(
    "vital_charttime"
)  # corrected variable name

# Step 2: Group by stay_id, subject_id, hadm_id and resample
grouped = vitals_filtered_1.groupby(["stay_id", "subject_id", "hadm_id"]).resample(
    "24H"
)

# Step 3: Calculate max and min of each vital sign
vitals_agg = grouped.agg(
    {
        "heart_rate": ["max", "min"],
        "sbp": ["max", "min"],
        "glucose": ["max", "min"],
        "temperature": ["max", "min"],
        "dbp_ni": ["max", "min"],
        "mbp": ["max", "min"],
        "sbp_ni": ["max", "min"],
        "dbp_ni": ["max", "min"],
        "mbp_ni": ["max", "min"],
    }
)

# Step 4: Reset the index
vitals_agg = vitals_agg.reset_index()

vitals_agg

stay_id subject_id   hadm_id vital_charttime heart_rate           sbp  \
                                                            max    min    max   
0      30002521   13269859  26734917      2161-05-18      101.0   90.0  149.0   
1      30002521   13269859  26734917      2161-05-19       98.0   82.0  161.0   
2      30002521   13269859  26734917      2161-05-20       90.0   76.0  158.0   
3      30004391   18730522  21463350      2153-09-05      112.0   91.0  132.0   
4      30004391   18730522  21463350      2153-09-06      158.0   88.0  114.0   
...         ...        ...       ...             ...        ...    ...    ...   
26897  39998622   19046950  24352151      2135-02-16      110.0  100.0  127.0   
26898  39998622   19046950  24352151      2135-02-17      104.0   84.0  121.0   
26899  39998622   19046950  24352151      2135-02-18       97.0   75.0  121.0   
26900  39998622   19046950  24352151      2135-02-19      102.0   82.0  117.0   
26901  39998622   19046950  24352151      2135-02-20      114.0   95.0  129.0   

             glucose        temperature        dbp_ni          mbp        \
         min     max    min         max    min    max   min    max   min   
0      116.0   344.0   62.0       36.67  36.67  110.0  62.0  118.0  73.0   
1       98.0   395.0   72.0       37.00  36.67   94.0  47.0  108.0  59.0   
2      123.0   212.0  166.0       36.72  36.56   89.0  75.0  107.0  82.0   
3       92.0   138.0  138.0       37.06  36.61   84.0  54.0   94.0  70.0   
4       82.0   107.0   86.0       37.11  36.50   73.0  54.0   81.0  60.0   
...      ...     ...    ...         ...    ...    ...   ...    ...   ...   
26897   89.0   138.0   92.0       39.00  36.11   79.0  57.0   90.0  64.0   
26898   92.0   175.0  145.0       39.28  37.67   73.0  56.0   82.0  66.0   
26899   85.0   164.0  125.0       39.04  37.22   78.0  44.0   83.0  54.0   
26900   89.0   158.0  137.0       38.40  36.89   84.0  52.0   89.0  59.0   
26901   94.0   259.0  161.0       37.67  36.89   94.0  54.0  100.0  66.0   

      sbp_ni        mbp_ni        
         max    min    max   min  
0      149.0  116.0  118.0  73.0  
1      161.0   98.0  108.0  59.0  
2      158.0  123.0  107.0  82.0  
3      132.0   92.0   94.0  70.0  
4      114.0   82.0   81.0  60.0  
...      ...    ...    ...   ...  
26897  127.0   89.0   90.0  64.0  
26898  121.0   92.0   82.0  66.0  
26899  121.0   85.0   83.0  54.0  
26900  117.0   89.0   89.0  59.0  
26901  129.0   94.0  100.0  66.0  

[26902 rows x 20 columns]

In [18]:
# Flatten the column MultiIndex
vitals_agg.columns = ["_".join(col).strip() for col in vitals_agg.columns.values]

# Rename columns to remove the trailing underscore
vitals_agg.columns = vitals_agg.columns.str.rstrip("_")

vitals_agg

,stay_id,subject_id,hadm_id,vital_charttime,heart_rate_max,heart_rate_min,sbp_max,sbp_min,glucose_max,glucose_min,temperature_max,temperature_min,dbp_ni_max,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min
0,30002521,13269859,26734917,2161-05-18,101.0,90.0,149.0,116.0,344.0,62.0,36.67,36.67,110.0,62.0,118.0,73.0,149.0,116.0,118.0,73.0
1,30002521,13269859,26734917,2161-05-19,98.0,82.0,161.0,98.0,395.0,72.0,37.00,36.67,94.0,47.0,108.0,59.0,161.0,98.0,108.0,59.0
2,30002521,13269859,26734917,2161-05-20,90.0,76.0,158.0,123.0,212.0,166.0,36.72,36.56,89.0,75.0,107.0,82.0,158.0,123.0,107.0,82.0
3,30004391,18730522,21463350,2153-09-05,112.0,91.0,132.0,92.0,138.0,138.0,37.06,36.61,84.0,54.0,94.0,70.0,132.0,92.0,94.0,70.0
4,30004391,18730522,21463350,2153-09-06,158.0,88.0,114.0,82.0,107.0,86.0,37.11,36.50,73.0,54.0,81.0,60.0,114.0,82.0,81.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26897,39998622,19046950,24352151,2135-02-16,110.0,100.0,127.0,89.0,138.0,92.0,39.00,36.11,79.0,57.0,90.0,64.0,127.0,89.0,90.0,64.0
26898,39998622,19046950,24352151,2135-02-17,104.0,84.0,121.0,92.0,175.0,145.0,39.28,37.67,73.0,56.0,82.0,66.0,121.0,92.0,82.0,66.0
26899,39998622,19046950,24352151,2135-02-18,97.0,75.0,121.0,85.0,164.0,125.0,39.04,37.22,78.0,44.0,83.0,54.0,121.0,85.0,83.0,54.0
26900,39998622,19046950,24352151,2135-02-19,102.0,82.0,117.0,89.0,158.0,137.0,38.40,36.89,84.0,52.0,89.0,59.0,117.0,89.0,89.0,59.0


In [19]:
# unique stay ids from vitals_agg

unique_stay_ids = vitals_agg["stay_id"].unique()

In [20]:
# do the same for urine

urine = urine.rename(columns={"charttime": "urine_charttime"})

urine_filtered = urine[urine["stay_id"].isin(unique_stay_ids)]

urine_filtered = urine_filtered.sort_values(
    by=["urine_charttime", "stay_id"], ascending=True
)

urine_filtered = urine_filtered.dropna(subset=["urineoutput"])

urine_filtered

,stay_id,urine_charttime,urineoutput
2206659,30927169,2110-01-21 19:01:00,175.0
824787,30927169,2110-01-21 19:10:00,175.0
2795557,30927169,2110-01-21 20:00:00,115.0
2586970,30927169,2110-01-21 21:00:00,50.0
3104120,30927169,2110-01-21 22:00:00,30.0
...,...,...,...
2218643,35859492,2210-09-15 11:00:00,28.0
2465791,35859492,2210-09-15 17:00:00,150.0
324559,35859492,2210-09-15 20:00:00,20.0
878003,35859492,2210-09-15 23:00:00,20.0


In [21]:
# do the 24 hour resampling for urine

urine_filtered_1 = urine_filtered

urine_filtered_1["urine_charttime"] = pd.to_datetime(
    urine_filtered_1["urine_charttime"]
)

urine_filtered_1 = urine_filtered_1.set_index("urine_charttime")

grouped_urine = urine_filtered_1.groupby("stay_id").resample("24H")

urine_agg = grouped_urine.agg({"urineoutput": "sum"})

urine_agg = urine_agg.reset_index()

urine_agg

,stay_id,urine_charttime,urineoutput
0,30002521,2161-05-18,1450.0
1,30002521,2161-05-19,2450.0
2,30004391,2153-09-05,660.0
3,30004391,2153-09-06,1115.0
4,30004391,2153-09-07,1950.0
...,...,...,...
23834,39998622,2135-02-16,3895.0
23835,39998622,2135-02-17,2690.0
23836,39998622,2135-02-18,1950.0
23837,39998622,2135-02-19,3170.0


In [22]:
# join vitals_agg and urine_agg on stay_id and urine_charttime = vital_charttime

# Merge vitals_agg and urine_agg on stay_id and charttime
# Reset the index of vitals_agg
vitals_agg_reset = vitals_agg.reset_index()

# Merge vitals_agg and urine_agg on stay_id and charttime
merged_urine_vitals = pd.merge(
    vitals_agg_reset,
    urine_agg,
    left_on=["stay_id", "vital_charttime"],
    right_on=["stay_id", "urine_charttime"],
    how="inner",
)
merged_urine_vitals

,index,stay_id,subject_id,hadm_id,vital_charttime,heart_rate_max,heart_rate_min,sbp_max,sbp_min,glucose_max,...,dbp_ni_max,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min,urine_charttime,urineoutput
0,0,30002521,13269859,26734917,2161-05-18,101.0,90.0,149.0,116.0,344.0,...,110.0,62.0,118.0,73.0,149.0,116.0,118.0,73.0,2161-05-18,1450.0
1,1,30002521,13269859,26734917,2161-05-19,98.0,82.0,161.0,98.0,395.0,...,94.0,47.0,108.0,59.0,161.0,98.0,108.0,59.0,2161-05-19,2450.0
2,3,30004391,18730522,21463350,2153-09-05,112.0,91.0,132.0,92.0,138.0,...,84.0,54.0,94.0,70.0,132.0,92.0,94.0,70.0,2153-09-05,660.0
3,4,30004391,18730522,21463350,2153-09-06,158.0,88.0,114.0,82.0,107.0,...,73.0,54.0,81.0,60.0,114.0,82.0,81.0,60.0,2153-09-06,1115.0
4,5,30004391,18730522,21463350,2153-09-07,155.0,77.0,116.0,90.0,71.0,...,90.0,58.0,96.0,66.0,116.0,90.0,96.0,66.0,2153-09-07,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23805,26897,39998622,19046950,24352151,2135-02-16,110.0,100.0,127.0,89.0,138.0,...,79.0,57.0,90.0,64.0,127.0,89.0,90.0,64.0,2135-02-16,3895.0
23806,26898,39998622,19046950,24352151,2135-02-17,104.0,84.0,121.0,92.0,175.0,...,73.0,56.0,82.0,66.0,121.0,92.0,82.0,66.0,2135-02-17,2690.0
23807,26899,39998622,19046950,24352151,2135-02-18,97.0,75.0,121.0,85.0,164.0,...,78.0,44.0,83.0,54.0,121.0,85.0,83.0,54.0,2135-02-18,1950.0
23808,26900,39998622,19046950,24352151,2135-02-19,102.0,82.0,117.0,89.0,158.0,...,84.0,52.0,89.0,59.0,117.0,89.0,89.0,59.0,2135-02-19,3170.0


In [23]:
# sofa
sofa = pd.read_csv("sofa.csv")

In [24]:
sofa

,hadm_id,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,22069991,32374495,519,2135-10-10 19:00:00,2135-10-10 20:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,4,1,2,1,4,3,15
1,21266774,30207861,231,2165-04-15 11:00:00,2165-04-15 12:00:00,NaN,NaN,NaN,0.090035,NaN,...,3.0,0.0,NaN,3,2,4,4,0,3,16
2,29021241,34969901,47,2154-04-08 17:00:00,2154-04-08 18:00:00,NaN,NaN,NaN,0.437115,NaN,...,4.0,NaN,NaN,4,0,4,4,1,4,17
3,23059383,38506743,61,2147-10-31 10:00:00,2147-10-31 11:00:00,NaN,NaN,NaN,0.500345,NaN,...,4.0,NaN,4.0,3,2,1,4,2,4,16
4,28974251,34784026,41,2159-07-25 07:00:00,2159-07-25 08:00:00,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,4,2,4,1,0,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043897,20142342,32166897,37,2111-12-09 08:00:00,2111-12-09 09:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,3,2,4,0,3,2,14
6043898,27124643,37836499,30,2149-06-19 16:00:00,2149-06-19 17:00:00,NaN,NaN,NaN,0.079988,NaN,...,3.0,NaN,4.0,0,3,2,4,1,4,14
6043899,20433528,32896891,172,2189-06-17 16:00:00,2189-06-17 17:00:00,NaN,NaN,NaN,0.281897,NaN,...,4.0,NaN,NaN,2,4,4,4,0,0,14
6043900,20683707,35293872,133,2120-08-13 11:00:00,2120-08-13 12:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,3,4,3,0,3,1,14


In [25]:
sofa = sofa.rename(columns={"starttime": "sofa_starttime"})

sofa_filtered = sofa[sofa["stay_id"].isin(unique_stay_ids)]

sofa_filtered = sofa_filtered.sort_values(
    by=["sofa_starttime", "stay_id"], ascending=True
)

sofa_filtered

,hadm_id,stay_id,hr,sofa_starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
3221817,22277686,30927169,0,2110-01-21 18:00:00,2110-01-21 19:00:00,NaN,696.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0,1,2,0,0,0,3
1273647,22277686,30927169,1,2110-01-21 19:00:00,2110-01-21 20:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,1,2,0,0,0,3
2042776,22277686,30927169,2,2110-01-21 20:00:00,2110-01-21 21:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,1,2,0,0,0,3
5142574,22277686,30927169,3,2110-01-21 21:00:00,2110-01-21 22:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,1,2,0,0,0,3
2266340,22277686,30927169,4,2110-01-21 22:00:00,2110-01-21 23:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,1,2,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4346015,22241351,35859492,214,2210-09-15 21:00:00,2210-09-15 22:00:00,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,1,4,3,8
1595273,22241351,35859492,215,2210-09-15 22:00:00,2210-09-15 23:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,0,0,0,1,4,3,8
482512,22241351,35859492,216,2210-09-15 23:00:00,2210-09-16 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,1,4,3,8
1042583,22241351,35859492,217,2210-09-16 00:00:00,2210-09-16 01:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,0,0,0,1,4,3,8


In [26]:
# take max of each sofa score for each stay_id and resample by 24 hours

sofa_filtered_1 = sofa_filtered

sofa_filtered_1["sofa_starttime"] = pd.to_datetime(sofa_filtered_1["sofa_starttime"])

sofa_filtered_1 = sofa_filtered_1.set_index("sofa_starttime")

grouped_sofa = sofa_filtered_1.groupby("stay_id").resample("24H")

sofa_agg = grouped_sofa.agg(
    {
        "respiration_24hours": "max",
        "coagulation_24hours": "max",
        "liver_24hours": "max",
        "cardiovascular_24hours": "max",
        "cns_24hours": "max",
        "renal_24hours": "max",
        "sofa_24hours": "max",
    }
)

sofa_agg = sofa_agg.reset_index()

sofa_agg

# order by sofa_24hours max

,stay_id,sofa_starttime,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,30002521,2161-05-18,0,0,0,0,0,0,0
1,30002521,2161-05-19,0,0,0,1,0,0,1
2,30002521,2161-05-20,0,0,0,1,0,0,1
3,30004391,2153-09-05,0,0,0,4,0,1,5
4,30004391,2153-09-06,0,1,0,4,0,1,6
...,...,...,...,...,...,...,...,...,...
26376,39998622,2135-02-16,2,0,0,1,0,0,3
26377,39998622,2135-02-17,2,0,0,1,1,0,3
26378,39998622,2135-02-18,1,0,0,1,1,0,3
26379,39998622,2135-02-19,0,0,0,1,0,0,1


In [27]:
# Merge vitals_agg and urine_agg on stay_id and charttime

In [28]:
# merge sofa and merged_urine_vitals on stay_id and sofa_starttime = vital_charttime

# Merge sofa_agg and merged_urine_vitals on stay_id and charttime

# Reset the index of sofa_agg

sofa_agg_reset = sofa_agg.reset_index()

# Merge sofa_agg and merged_urine_vitals on stay_id and charttime

merged_sofa_vitals_urine = pd.merge(
    sofa_agg_reset,
    merged_urine_vitals,
    left_on=["stay_id", "sofa_starttime"],
    right_on=["stay_id", "vital_charttime"],
    how="inner",
)

In [29]:
merged_sofa_vitals_urine

,index_x,stay_id,sofa_starttime,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,...,dbp_ni_max,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min,urine_charttime,urineoutput
0,0,30002521,2161-05-18,0,0,0,0,0,0,0,...,110.0,62.0,118.0,73.0,149.0,116.0,118.0,73.0,2161-05-18,1450.0
1,1,30002521,2161-05-19,0,0,0,1,0,0,1,...,94.0,47.0,108.0,59.0,161.0,98.0,108.0,59.0,2161-05-19,2450.0
2,3,30004391,2153-09-05,0,0,0,4,0,1,5,...,84.0,54.0,94.0,70.0,132.0,92.0,94.0,70.0,2153-09-05,660.0
3,4,30004391,2153-09-06,0,1,0,4,0,1,6,...,73.0,54.0,81.0,60.0,114.0,82.0,81.0,60.0,2153-09-06,1115.0
4,5,30004391,2153-09-07,0,1,0,1,1,0,3,...,90.0,58.0,96.0,66.0,116.0,90.0,96.0,66.0,2153-09-07,1950.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23783,26376,39998622,2135-02-16,2,0,0,1,0,0,3,...,79.0,57.0,90.0,64.0,127.0,89.0,90.0,64.0,2135-02-16,3895.0
23784,26377,39998622,2135-02-17,2,0,0,1,1,0,3,...,73.0,56.0,82.0,66.0,121.0,92.0,82.0,66.0,2135-02-17,2690.0
23785,26378,39998622,2135-02-18,1,0,0,1,1,0,3,...,78.0,44.0,83.0,54.0,121.0,85.0,83.0,54.0,2135-02-18,1950.0
23786,26379,39998622,2135-02-19,0,0,0,1,0,0,1,...,84.0,52.0,89.0,59.0,117.0,89.0,89.0,59.0,2135-02-19,3170.0


In [30]:
filtered_df = pd.read_csv("filtered_final.csv")

In [31]:
filtered_df

,Unnamed: 0,hadm_id,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,...,insurance,language,marital_status,race,gender,hospital_expire_flag,los_hospital,admission_age,starttime_y,endtime
0,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,F,0,21,66,2158-05-19 18:00:00,2158-05-20 14:00:00
1,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500.0,17.8,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
2,16,20024229.0,16779215-136,16779215,2132-08-31 00:37:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
3,17,20024229.0,16779215-145,16779215,2132-08-31 07:56:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
4,18,20024229.0,16779215-164,16779215,2132-08-31 16:00:00,1500.0,24.3,2132-09-02 13:45:00,0.6,0,...,Other,ENGLISH,DIVORCED,WHITE,M,0,12,65,2132-08-27 16:00:00,2132-09-01 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5763,13108,25265899.0,13470702-63,13470702,2134-02-11 03:27:00,1000.0,8.0,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
5764,13109,25265899.0,13470702-81,13470702,2134-02-11 12:28:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
5765,13110,25265899.0,13470702-97,13470702,2134-02-11 20:15:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,Other,ENGLISH,SINGLE,WHITE,F,0,15,27,2134-02-10 18:00:00,2134-02-14 03:00:00
5766,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000.0,11.8,2150-12-07 03:56:00,4.6,3,...,Other,ENGLISH,NaN,UNKNOWN,M,1,2,23,2150-12-05 04:00:00,2150-12-08 12:00:00


In [32]:
merged_sofa_vitals_urine[merged_sofa_vitals_urine["subject_id"] == 14104652]

,index_x,stay_id,sofa_starttime,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours,...,dbp_ni_max,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min,urine_charttime,urineoutput
12766,14304,35461437,2158-03-18,0,0,0,0,0,0,0,...,81.0,81.0,97.0,92.0,135.0,112.0,97.0,92.0,2158-03-18,850.0
12767,14305,35461437,2158-03-19,0,0,0,0,1,0,1,...,101.0,56.0,108.0,74.0,123.0,101.0,108.0,74.0,2158-03-19,2100.0
12768,14306,35461437,2158-03-20,0,0,0,0,1,0,1,...,83.0,67.0,94.0,74.0,139.0,87.0,94.0,74.0,2158-03-20,575.0
21560,23902,39090944,2158-05-19,2,1,0,4,0,0,7,...,58.0,58.0,120.0,65.0,78.0,78.0,65.0,65.0,2158-05-19,605.0
21561,23903,39090944,2158-05-20,2,2,0,4,0,3,11,...,NaN,NaN,93.0,61.0,NaN,NaN,NaN,NaN,2158-05-20,450.0
21562,23904,39090944,2158-05-21,2,3,0,4,0,3,11,...,82.0,61.0,96.0,67.0,138.0,98.0,96.0,70.0,2158-05-21,605.0
21563,23905,39090944,2158-05-22,2,3,0,3,1,3,12,...,79.0,61.0,93.0,74.0,132.0,97.0,93.0,74.0,2158-05-22,960.0
21564,23906,39090944,2158-05-23,0,2,0,0,1,0,3,...,84.0,55.0,101.0,70.0,145.0,99.0,101.0,70.0,2158-05-23,1950.0
21565,23907,39090944,2158-05-24,0,2,0,0,0,0,2,...,94.0,71.0,111.0,87.0,165.0,122.0,111.0,87.0,2158-05-24,2200.0


In [35]:
# save the merged_sofa_vitals_urine to a csv file

merged_sofa_vitals_urine.to_csv("merged_sofa_vitals_urine.csv", index=False)

In [36]:
filtered_df["emar_charttime_day"] = pd.to_datetime(
    filtered_df["emar_charttime"]
).dt.date

In [37]:
# Convert to datetime
filtered_df["emar_charttime_day"] = pd.to_datetime(filtered_df["emar_charttime_day"])
merged_sofa_vitals_urine["sofa_starttime"] = pd.to_datetime(
    merged_sofa_vitals_urine["sofa_starttime"]
)

# Perform inner join
merged_df = pd.merge(
    filtered_df, merged_sofa_vitals_urine, on="subject_id", how="inner"
)

# Add new column 'days' which is the difference between 'emar_charttime_day' and 'sofa_starttime'
merged_df["days"] = (
    merged_df["sofa_starttime"] - merged_df["emar_charttime_day"]
).dt.days

merged_df

,Unnamed: 0,hadm_id_x,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,...,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min,urine_charttime,urineoutput,days
0,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,81.0,97.0,92.0,135.0,112.0,97.0,92.0,2158-03-18,850.0,-63
1,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,56.0,108.0,74.0,123.0,101.0,108.0,74.0,2158-03-19,2100.0,-62
2,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,67.0,94.0,74.0,139.0,87.0,94.0,74.0,2158-03-20,575.0,-61
3,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,58.0,120.0,65.0,78.0,78.0,65.0,65.0,2158-05-19,605.0,-1
4,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,NaN,93.0,61.0,NaN,NaN,NaN,NaN,2158-05-20,450.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147453,13128,28203582.0,12701519-472,12701519,2190-09-30 22:09:00,1500.0,7.0,2190-10-02 12:09:00,0.8,1,...,52.0,84.0,62.0,122.0,94.0,84.0,62.0,2185-07-15,2700.0,-1903
147454,13128,28203582.0,12701519-472,12701519,2190-09-30 22:09:00,1500.0,7.0,2190-10-02 12:09:00,0.8,1,...,53.0,85.0,64.0,120.0,98.0,85.0,64.0,2185-07-16,3060.0,-1902
147455,13128,28203582.0,12701519-472,12701519,2190-09-30 22:09:00,1500.0,7.0,2190-10-02 12:09:00,0.8,1,...,52.0,86.0,61.0,121.0,91.0,86.0,61.0,2185-07-17,3000.0,-1901
147456,13128,28203582.0,12701519-472,12701519,2190-09-30 22:09:00,1500.0,7.0,2190-10-02 12:09:00,0.8,1,...,58.0,94.0,67.0,125.0,99.0,94.0,67.0,2185-07-18,2550.0,-1900


In [38]:
# filter for rows where days is 0 or 1

merged_df = merged_df[(merged_df["days"] == 0) | (merged_df["days"] == 1)]

merged_df

,Unnamed: 0,hadm_id_x,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,...,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min,urine_charttime,urineoutput,days
4,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,NaN,93.0,61.0,NaN,NaN,NaN,NaN,2158-05-20,450.0,0
5,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,61.0,96.0,67.0,138.0,98.0,96.0,70.0,2158-05-21,605.0,1
12,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500.0,17.8,2132-08-31 14:50:00,0.6,0,...,22.0,105.0,43.0,180.0,87.0,105.0,43.0,2132-08-30,1460.0,0
13,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500.0,17.8,2132-08-31 14:50:00,0.6,0,...,44.0,81.0,58.0,144.0,103.0,81.0,58.0,2132-08-31,3561.0,1
20,16,20024229.0,16779215-136,16779215,2132-08-31 00:37:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,44.0,81.0,58.0,144.0,103.0,81.0,58.0,2132-08-31,3561.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147399,13110,25265899.0,13470702-97,13470702,2134-02-11 20:15:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,64.0,108.0,76.0,142.0,101.0,96.0,77.0,2134-02-12,3095.0,1
147413,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000.0,11.8,2150-12-07 03:56:00,4.6,3,...,54.0,135.0,60.0,174.0,97.0,135.0,66.0,2150-12-05,1247.0,0
147414,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000.0,11.8,2150-12-07 03:56:00,4.6,3,...,55.0,100.0,65.0,145.0,101.0,100.0,66.0,2150-12-06,2390.0,1
147424,13128,28203582.0,12701519-472,12701519,2190-09-30 22:09:00,1500.0,7.0,2190-10-02 12:09:00,0.8,1,...,NaN,76.0,65.0,NaN,NaN,NaN,NaN,2190-09-30,841.0,0


In [42]:
day_0 = merged_df[merged_df["days"] == 0]

day_0

,Unnamed: 0,hadm_id_x,emar_id,subject_id,emar_charttime,dose_given,vanco_lab_mean_next48h,creat_charttime,creat,aki_stage_creat,...,dbp_ni_min,mbp_max,mbp_min,sbp_ni_max,sbp_ni_min,mbp_ni_max,mbp_ni_min,urine_charttime,urineoutput,days
4,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,NaN,93.0,61.0,NaN,NaN,NaN,NaN,2158-05-20,450.0,0
12,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500.0,17.8,2132-08-31 14:50:00,0.6,0,...,22.0,105.0,43.0,180.0,87.0,105.0,43.0,2132-08-30,1460.0,0
20,16,20024229.0,16779215-136,16779215,2132-08-31 00:37:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,44.0,81.0,58.0,144.0,103.0,81.0,58.0,2132-08-31,3561.0,0
27,17,20024229.0,16779215-145,16779215,2132-08-31 07:56:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,44.0,81.0,58.0,144.0,103.0,81.0,58.0,2132-08-31,3561.0,0
34,18,20024229.0,16779215-164,16779215,2132-08-31 16:00:00,1500.0,24.3,2132-09-02 13:45:00,0.6,0,...,44.0,81.0,58.0,144.0,103.0,81.0,58.0,2132-08-31,3561.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147366,13108,25265899.0,13470702-63,13470702,2134-02-11 03:27:00,1000.0,8.0,2134-02-12 23:20:00,0.7,0,...,58.0,181.0,70.0,113.0,98.0,89.0,70.0,2134-02-11,1560.0,0
147382,13109,25265899.0,13470702-81,13470702,2134-02-11 12:28:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,58.0,181.0,70.0,113.0,98.0,89.0,70.0,2134-02-11,1560.0,0
147398,13110,25265899.0,13470702-97,13470702,2134-02-11 20:15:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,58.0,181.0,70.0,113.0,98.0,89.0,70.0,2134-02-11,1560.0,0
147413,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000.0,11.8,2150-12-07 03:56:00,4.6,3,...,54.0,135.0,60.0,174.0,97.0,135.0,66.0,2150-12-05,1247.0,0


In [43]:
# now, for each row, if the day is 1, add those values as columns to the previous row with _48 suffix

# Create a new DataFrame to store the final result

# get a df with only rows where days is 1

day_1 = merged_df[merged_df["days"] == 1]

day_1

# add _48 suffix to the columns

day_1.columns = [col + "_48" for col in day_1.columns]

day_1

,Unnamed: 0_48,hadm_id_x_48,emar_id_48,subject_id_48,emar_charttime_48,dose_given_48,vanco_lab_mean_next48h_48,creat_charttime_48,creat_48,aki_stage_creat_48,...,dbp_ni_min_48,mbp_max_48,mbp_min_48,sbp_ni_max_48,sbp_ni_min_48,mbp_ni_max_48,mbp_ni_min_48,urine_charttime_48,urineoutput_48,days_48
5,2,20011505.0,14104652-131,14104652,2158-05-20 10:53:00,1000.0,5.2,2158-05-20 12:10:00,1.2,2,...,61.0,96.0,67.0,138.0,98.0,96.0,70.0,2158-05-21,605.0,1
13,15,20024229.0,16779215-127,16779215,2132-08-30 17:17:00,1500.0,17.8,2132-08-31 14:50:00,0.6,0,...,44.0,81.0,58.0,144.0,103.0,81.0,58.0,2132-08-31,3561.0,1
21,16,20024229.0,16779215-136,16779215,2132-08-31 00:37:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,44.0,93.0,61.0,165.0,116.0,93.0,61.0,2132-09-01,4155.0,1
28,17,20024229.0,16779215-145,16779215,2132-08-31 07:56:00,1500.0,24.3,2132-08-31 14:50:00,0.6,0,...,44.0,93.0,61.0,165.0,116.0,93.0,61.0,2132-09-01,4155.0,1
35,18,20024229.0,16779215-164,16779215,2132-08-31 16:00:00,1500.0,24.3,2132-09-02 13:45:00,0.6,0,...,44.0,93.0,61.0,165.0,116.0,93.0,61.0,2132-09-01,4155.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147367,13108,25265899.0,13470702-63,13470702,2134-02-11 03:27:00,1000.0,8.0,2134-02-12 23:20:00,0.7,0,...,64.0,108.0,76.0,142.0,101.0,96.0,77.0,2134-02-12,3095.0,1
147383,13109,25265899.0,13470702-81,13470702,2134-02-11 12:28:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,64.0,108.0,76.0,142.0,101.0,96.0,77.0,2134-02-12,3095.0,1
147399,13110,25265899.0,13470702-97,13470702,2134-02-11 20:15:00,1000.0,9.2,2134-02-12 23:20:00,0.7,0,...,64.0,108.0,76.0,142.0,101.0,96.0,77.0,2134-02-12,3095.0,1
147414,13127,26369886.0,10676055-61,10676055,2150-12-05 19:38:00,1000.0,11.8,2150-12-07 03:56:00,4.6,3,...,55.0,100.0,65.0,145.0,101.0,100.0,66.0,2150-12-06,2390.0,1


In [48]:
# drop the days_48 column

# day_1 = day_1.drop(columns=['days_48'])

# now just merge day_0 and day_1 on subject_id and emar_charttime_48 = emar_charttime

# Merge day_0 and day_1 on subject_id and emar_charttime_48 = emar_charttime

final_df = pd.merge(
    day_0,
    day_1,
    left_on=["subject_id", "emar_charttime"],
    right_on=["subject_id_48", "emar_charttime_48"],
    how="left",
)

final_df

# remove urine_charttime_48 column

final_df = final_df.drop(columns=["urine_charttime_48"])

final_df.columns

Index(['Unnamed: 0', 'hadm_id_x', 'emar_id', 'subject_id', 'emar_charttime',
       'dose_given', 'vanco_lab_mean_next48h', 'creat_charttime', 'creat',
       'aki_stage_creat',
       ...
       'temperature_min_48', 'dbp_ni_max_48', 'dbp_ni_min_48', 'mbp_max_48',
       'mbp_min_48', 'sbp_ni_max_48', 'sbp_ni_min_48', 'mbp_ni_max_48',
       'mbp_ni_min_48', 'urineoutput_48'],
      dtype='object', length=108)

In [49]:
# save the final_df to a csv file

final_df.to_csv("final_df_modelling.csv", index=False)

In [51]:
# unique subject_ids from final_df

unique_subject_ids = final_df["subject_id"].unique()

len(unique_subject_ids)

1230